In [257]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [298]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import ExtraTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import Ridge

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer

In [316]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [299]:
df1 = pd.read_csv('./movies.csv')
df = df1.copy()

In [261]:
df

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,NaN,Drama,2020,"October 23, 2020 (United States)",3.1,18.0,Joseph Ebanks,Joseph Ebanks,Shannon Bond,United States,7000.0,NaN,NaN,90.0
7664,Dream Round,NaN,Comedy,2020,"February 7, 2020 (United States)",4.7,36.0,Dusty Dukatz,Lisa Huston,Michael Saquella,United States,NaN,NaN,Cactus Blue Entertainment,90.0
7665,Saving Mbango,NaN,Drama,2020,"April 27, 2020 (Cameroon)",5.7,29.0,Nkanya Nkwai,Lynno Lovert,Onyama Laura,United States,58750.0,NaN,Embi Productions,NaN
7666,It's Just Us,NaN,Drama,2020,"October 1, 2020 (United States)",NaN,NaN,James Randall,James Randall,Christina Roz,United States,15000.0,NaN,NaN,120.0


In [262]:
df = df.drop(labels =['released', 'company' ] , axis=1)
# Fill n/a values for numerical data with colomn mean 
df['score'].fillna(value=df['score'].mean() , inplace=True)
df['gross'].fillna(value=df['gross'].mean() , inplace=True)

# Fill n/a values for categorical data with 'unknown' word
df['rating'].fillna(value='unrated' , inplace=True)
df['writer'].fillna(value='unknown' , inplace=True)
df['star'].fillna(value='unknown' , inplace=True)
df['country'].fillna(value='unknown' , inplace=True)

# Delete lines with missing information 
df.dropna(subset=['votes'] , axis=0 , inplace=True)
df.dropna(subset=['budget'] , axis=0 , inplace=True)
df.dropna(subset=['runtime'] , axis=0 ,inplace=True)
# Check for duplicates
df.drop_duplicates(keep='first',inplace=True)
# Dropping gross values which is bigger than 2e9
outliers_indices = df.loc[df['gross']>2e9].index
df.drop(outliers_indices , inplace=True)

m1 = df.director.map(df.director.value_counts()) < 20
df.director =  df.director.mask(m1, 'other')

m2 = df.writer.map(df.writer.value_counts()) < 10
df.writer =  df.writer.mask(m2, 'other')

m3 = df.star.map(df.star.value_counts()) < 25
df.star =  df.star.mask(m3, 'other')

In [300]:
df = df.drop(labels =['released', 'company' ] , axis=1)
# Fill n/a values for numerical data with colomn mean 
df['score'].fillna(value=df['score'].mean() , inplace=True)
df['gross'].fillna(value=df['gross'].mean() , inplace=True)

# Delete lines with missing information 
df.dropna(subset=['votes'] , axis=0 , inplace=True)
df.dropna(subset=['budget'] , axis=0 , inplace=True)
df.dropna(subset=['runtime'] , axis=0 ,inplace=True)
df.dropna(subset=['rating'] , axis=0 , inplace=True)
df.dropna(subset=['writer'] , axis=0 , inplace=True)
df.dropna(subset=['star'] , axis=0 ,inplace=True)
df.dropna(subset=['country'] , axis=0 ,inplace=True)
# Check for duplicates
df.drop_duplicates(keep='first',inplace=True)
# Dropping gross values which is bigger than 2e9
outliers_indices = df.loc[df['gross']>2e9].index
df.drop(outliers_indices , inplace=True)

# m1 = df.director.map(df.director.value_counts()) < 20
# df.director =  df.director.mask(m1, 'other')

# m2 = df.writer.map(df.writer.value_counts()) < 10
# df.writer =  df.writer.mask(m2, 'other')

# m3 = df.star.map(df.star.value_counts()) < 25
# df.star =  df.star.mask(m3, 'other')

In [405]:
X, y = df.drop(['name', 'year', 'score'], axis = 1), df['score'] # removed low correlated ones

In [406]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [378]:
num = list(X_train.select_dtypes('number').columns)
cat = list(X_train.select_dtypes('category').columns)

model_scale = MinMaxScaler()
model_encoder = OneHotEncoder()
transformer = ColumnTransformer([('num',model_scale,num),
                                 ('cat',model_encoder,cat)])

mod_ada_2 = AdaBoostRegressor(ExtraTreeRegressor()) # base_estimator=None by default
model_combined = make_pipeline(transformer, mod_ada_2)

n_estimators = [int(x) for x in np.linspace(start = 1, stop = 30, num = 1)]
parameters = {'adaboostregressor__n_estimators': n_estimators
             }
gridCV= GridSearchCV(mod_ada_2, parameters, cv=10)
model_combined_optimal = GridSearchCV(model_combined,
          param_grid = parameters, n_jobs = -1, cv = 10, return_train_score=True)
model_combined_optimal = model_combined_optimal.fit(X_train, y_train)
best_model_extra = model_combined_optimal.best_estimator_
best_model_extra

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  ['votes', 'budget', 'gross',
                                                   'runtime']),
                                                 ('cat', OneHotEncoder(),
                                                  [])])),
                ('adaboostregressor',
                 AdaBoostRegressor(base_estimator=ExtraTreeRegressor(),
                                   n_estimators=1))])

In [370]:
best_model_extra.score(X_train,y_train)

0.5880897737689468

In [371]:
best_model_extra.score(X_test,y_test)

0.07580914277544637

#### Ridge regression

In [407]:
model_scale = MinMaxScaler()
model_ridge = Ridge()
model_encoder = OneHotEncoder()
num = list(X_train.select_dtypes('number').columns)
cat = list(X_train.select_dtypes('category').columns)
transformer = ColumnTransformer([('num',model_scale,num),
                                 ('cat',model_encoder,cat)])
model_combined = make_pipeline(transformer, model_ridge)
alpha_grid = np.linspace(0.0001, 50, 50)
model_combined_optimal = GridSearchCV(model_combined, 
                             param_grid = {'ridge__alpha': np.linspace(0.0001, 50, 50)})
model_combined_optimal.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         MinMaxScaler(),
                                                                         ['votes',
                                                                          'budget',
                                                                          'gross',
                                                                          'runtime']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         [])])),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': array([1.00000000e-04, 1.02050612e+00, 2.04091224e+00, 3.06131837e+00,
       4.08172449e+00, 5.10213061e+00, 6.12253673e+00, 7.1429428...
       2.44898469e+01, 2.55102531e+01, 2.65306592e+01, 2.75510653e+01,
       2.85714714e+01, 2.95918776e+01, 3.06122837e+01, 3.16326898e+01,
       3.26530959e+01, 3.36735020e+01, 3.46939082e+01, 3.57143143e+01,
       3.67347204e+01, 3.77551265e+01, 3.87755327e+01, 3.97959388e+01,
       4.08163449e+01, 4.18367510e+01, 4.28571571e+01, 4.38775633e+01,
       4.48979694e+01, 4.59183755e+01, 4.69387816e+01, 4.79591878e+01,
       4.89795939e+01, 5.00000000e+01])})

In [408]:
model_combined_optimal.best_params_

{'ridge__alpha': 0.0001}

In [409]:
model_combined_optimal.score(X_train,y_train)

0.3395803247727872

In [410]:
model_combined_optimal.score(X_test,y_test)

0.33073256988397715

#### Ridge regression reduced params

In [364]:
X, y = df.drop(['name', 'year', 'score', 'director', 'writer','star','country'], axis = 1), df['score'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [365]:
model_scale = MinMaxScaler()
model_ridge = Ridge()
model_encoder = OneHotEncoder()
num = list(X_train.select_dtypes('number').columns)
cat = list(X_train.select_dtypes('category').columns)
transformer = ColumnTransformer([('num',model_scale,num),
                                 ('cat',model_encoder,cat)])
model_combined = make_pipeline(transformer, model_ridge)
alpha_grid = np.linspace(0.0001, 50, 50)
model_combined_optimal = GridSearchCV(model_combined, 
                             param_grid = {'ridge__alpha': np.linspace(0.0001, 1, 50)})
model_combined_optimal.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         MinMaxScaler(),
                                                                         ['votes',
                                                                          'budget',
                                                                          'gross',
                                                                          'runtime']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         [])])),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': array([1.00000000e-04, 2.05061224e-02, 4.09122449e-02, 6.13183673e-02,
       8.17244898e-02, 1.02130612e-01, 1.22536735e-01, 1.4294285...
       4.89846939e-01, 5.10253061e-01, 5.30659184e-01, 5.51065306e-01,
       5.71471429e-01, 5.91877551e-01, 6.12283673e-01, 6.32689796e-01,
       6.53095918e-01, 6.73502041e-01, 6.93908163e-01, 7.14314286e-01,
       7.34720408e-01, 7.55126531e-01, 7.75532653e-01, 7.95938776e-01,
       8.16344898e-01, 8.36751020e-01, 8.57157143e-01, 8.77563265e-01,
       8.97969388e-01, 9.18375510e-01, 9.38781633e-01, 9.59187755e-01,
       9.79593878e-01, 1.00000000e+00])})

In [366]:
model_combined_optimal.best_params_

{'ridge__alpha': 0.32659795918367346}

In [367]:
model_combined_optimal.score(X_train,y_train)

0.3292590820484762

In [368]:
model_combined_optimal.score(X_test,y_test)

0.37260524723106425

In [372]:
model_scale = MinMaxScaler()
model = KNeighborsRegressor()
model_encoder = OneHotEncoder()
num = list(X_train.select_dtypes('number').columns)
cat = list(X_train.select_dtypes('category').columns)
transformer = ColumnTransformer([('num',model_scale,num),
                                 ('cat',model_encoder,cat)])
model_combined = make_pipeline(transformer, model)
n_neighbors_grid = np.arange(1,51,2)
model_optimal = GridSearchCV(model_combined,
param_grid = {'kneighborsregressor__n_neighbors': n_neighbors_grid},
        cv = 10, return_train_score=True)
model_optimal.fit(X_train,y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         MinMaxScaler(),
                                                                         ['votes',
                                                                          'budget',
                                                                          'gross',
                                                                          'runtime']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         [])])),
                                       ('kneighborsregressor',
                                        KNeighborsRegressor())]),
             param_grid={'kneighborsregressor__n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33,
       35, 37, 39, 41, 43, 45, 47, 49])},
             return_train_score=True)

In [373]:
model_combined_optimal.best_params_

{'adaboostregressor__n_estimators': 1}

In [374]:
model_combined_optimal.score(X_train,y_train)

0.5880897737689468

In [375]:
model_combined_optimal.score(X_test,y_test)

0.07580914277544637